In [1]:
from pathlib import Path
from collections import deque
import pandas as pd
from collections import defaultdict

def load_file(path):
    res = []
    start = None
    with Path(path).open() as f:
        for line in f.readlines():
            res.append(list(line.strip()))
    return res

next_dirs = {
    ".": ((0, -1), (1, 0), (0, 1), (-1, 0)),
    "#": (),
    ">": ((0, 1),),
    "v": ((1, 0),),
    "<": ((0, -1),),
}

def find_way(grid):
    m = len(grid)
    n = len(grid[0])
    res = 0
    queue = deque([(0, 0, 1, set())])
    max_step = 0
    while queue:
        steps, i, j, seen = queue.popleft()
        if i == m-1:
            res = max(res, steps)
            continue
        ds = next_dirs[grid[i][j]]
        new_seen = seen.copy()
        new_seen.add((i, j))
        for di, dj in ds:
            ni = i + di
            nj = j + dj
            if 0 <= ni < m and 0 <= nj < n and grid[ni][nj] != "#"  and (ni, nj) not in seen:
                queue.append((steps+1, ni, nj, new_seen))
    return res

In [2]:
grid = load_file("23_test.txt")
find_way(grid)

94

In [3]:
pd.DataFrame(grid)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,#,.,#,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
1,#,.,.,.,.,.,.,.,#,#,...,#,#,#,#,.,.,.,#,#,#
2,#,#,#,#,#,#,#,.,#,#,...,#,#,#,#,.,#,.,#,#,#
3,#,#,#,.,.,.,.,.,#,.,...,.,#,#,#,.,#,.,#,#,#
4,#,#,#,v,#,#,#,#,#,.,...,.,#,#,#,.,#,.,#,#,#
5,#,#,#,.,>,.,.,.,#,.,...,.,.,.,.,.,#,.,.,.,#
6,#,#,#,v,#,#,#,.,#,.,...,#,#,#,#,#,#,#,#,.,#
7,#,#,#,.,.,.,#,.,#,.,...,.,.,.,.,.,#,.,.,.,#
8,#,#,#,#,#,.,#,.,#,.,...,#,#,#,#,.,#,.,#,#,#
9,#,.,.,.,.,.,#,.,#,.,...,.,.,.,.,.,#,.,.,.,#


In [4]:
grid = load_file("23_input.txt")  
find_way(grid)  # 1998

1998

In [5]:
def sparse(grid):
    graph = defaultdict(dict)
    m = len(grid)
    n = len(grid[0])

    queue = deque([((0, 1), (0, 1), 0, set())])
    while queue:
        start, cur, cost, seen = queue.popleft()
        i, j = cur
        if i == n - 1:
            graph[start][(i, j)] = cost
            continue
        nex = set()
        for di, dj in ((0, 1), (1, 0), (0, -1), (-1, 0)):
            ni = i + di
            nj = j + dj
            if (
                0 <= ni < m
                and 0 <= nj < n
                and grid[ni][nj] != "#"
                and (ni, nj) not in seen
            ):
                nex.add((ni, nj))
        if len(nex) == 1:
            queue.append((start, nex.pop(), cost + 1, seen | {(i, j)}))
        elif len(nex) > 1:
            graph[start][(i, j)] = cost
            if (i, j) not in graph:
                for ni, nj in nex:
                    queue.append(((i, j), (ni, nj), 1, {(i, j)}))
            graph[(i, j)][start] = cost        

    return graph

In [6]:
grid = load_file("23_test.txt")
graph = sparse(grid)
graph

defaultdict(dict,
            {(0, 1): {(5, 3): 15},
             (5, 3): {(0, 1): 15, (3, 11): 22, (13, 5): 22},
             (3, 11): {(5, 3): 22, (13, 13): 24, (11, 21): 30},
             (13, 5): {(5, 3): 22, (13, 13): 12, (19, 13): 38},
             (13, 13): {(13, 5): 12, (19, 13): 10, (3, 11): 24, (11, 21): 18},
             (19, 13): {(13, 13): 10, (19, 19): 10, (13, 5): 38},
             (11, 21): {(3, 11): 30, (13, 13): 18, (19, 19): 10},
             (19, 19): {(19, 13): 10, (22, 21): 5, (11, 21): 10}})

In [7]:
def find_way_sparse(graph, target):
    queue = deque([((0, 1), 0, set())])
    res = 0
    while queue:
        cur, c, seen = queue.popleft()
        for nxt in graph[cur]:
            if nxt not in seen:
                if nxt == target:
                    res = max(res, c + graph[cur][nxt])
                else:
                    queue.append((nxt, c + graph[cur][nxt], seen | {cur}))
    return res

In [8]:
grid = load_file("23_test.txt")
graph = sparse(grid)
find_way_sparse(graph, (len(grid)-1, len(grid[0])-2))  # 154

154

In [9]:
grid = load_file("23_input.txt")
graph = sparse(grid)
find_way_sparse(graph, (len(grid)-1, len(grid[0])-2))  # 6434

6434